<a href="https://colab.research.google.com/github/Kovasluksas/Kursinis-darbas/blob/main/kursinis%20darbas%20final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import unittest
from abc import ABC, abstractmethod

class StackInterface(ABC):
    @abstractmethod
    def push(self, item):
        pass

    @abstractmethod
    def pop(self):
        pass

    @abstractmethod
    def peek(self):
        pass

    @abstractmethod
    def is_empty(self):
        pass

    @abstractmethod
    def size(self):
        pass

    @abstractmethod
    def clear(self):
        pass

#Encapsulation
class Stack(StackInterface):
    def __init__(self):
        self._items = []

    def push(self, item):
        """Add an item to the top of the stack."""
        self._items.append(item)

    def pop(self):
        """Remove and return the top item from the stack."""
        if not self.is_empty():
            return self._items.pop()
        raise IndexError("Pop from empty stack")

    def peek(self):
        """Return the top item without removing it."""
        if not self.is_empty():
            return self._items[-1]
        raise IndexError("Peek from empty stack")

    def is_empty(self):
        """Check if the stack is empty."""
        return len(self._items) == 0

    def size(self):
        """Return the number of items in the stack."""
        return len(self._items)

    def clear(self):
        """Clear all items from the stack."""
        self._items = []

    def __str__(self):
        """String representation of the stack."""
        return f"Stack({self._items})"


#Composition
class FileHandler:
    def __init__(self, filename="stack_data.txt"):
        self.filename = filename

    def save_stack(self, stack):
        """Save stack contents to file."""
        with open(self.filename, 'w') as f:
            for item in stack._items:
                f.write(f"{item}\n")

    def load_stack(self, stack):
        """Load stack contents from file."""
        if not os.path.exists(self.filename):
            return

        stack.clear()
        with open(self.filename, 'r') as f:
            for line in f:
                item = line.strip()
                try:
                    item = float(item) if '.' in item else int(item)
                except ValueError:
                    pass
                stack.push(item)

#Factory Method
class StackFactory:
    @staticmethod
    def create_stack():
        """Factory method to create a new stack."""
        return Stack()
#Inheritance
class ExtendedStack(Stack):
    def __init__(self):
        super().__init__()
        self.file_handler = FileHandler()  # Composition

    def save_to_file(self):
        """Save stack to file."""
        self.file_handler.save_stack(self)

    def load_from_file(self):
        """Load stack from file."""
        self.file_handler.load_stack(self)
#Unit tests
class TestStack(unittest.TestCase):
    def setUp(self):
        self.stack = Stack()

    def test_push_pop(self):
        self.stack.push(1)
        self.stack.push(2)
        self.assertEqual(self.stack.pop(), 2)
        self.assertEqual(self.stack.pop(), 1)

    def test_peek(self):
        self.stack.push(3)
        self.assertEqual(self.stack.peek(), 3)
        self.assertEqual(self.stack.size(), 1)

    def test_is_empty(self):
        self.assertTrue(self.stack.is_empty())
        self.stack.push(1)
        self.assertFalse(self.stack.is_empty())

    def test_pop_empty(self):
        with self.assertRaises(IndexError):
            self.stack.pop()

    def test_peek_empty(self):
        with self.assertRaises(IndexError):
            self.stack.peek()

    def test_clear(self):
        self.stack.push(1)
        self.stack.push(2)
        self.stack.clear()
        self.assertTrue(self.stack.is_empty())
        self.assertEqual(self.stack.size(), 0)


class TestFileHandler(unittest.TestCase):
    def setUp(self):
        self.test_file = "test_stack.txt"
        self.file_handler = FileHandler(self.test_file)
        self.stack = Stack()

    def tearDown(self):
        if os.path.exists(self.test_file):
            os.remove(self.test_file)

    def test_save_load(self):
        self.stack.push(1)
        self.stack.push("two")
        self.stack.push(3.0)
        self.file_handler.save_stack(self.stack)

        new_stack = Stack()
        self.file_handler.load_stack(new_stack)

        self.assertEqual(new_stack.pop(), 3.0)
        self.assertEqual(new_stack.pop(), "two")
        self.assertEqual(new_stack.pop(), 1)


def main():
    print("Stack Class Demonstration")
    print("1. Create a new stack")
    print("2. Push an item")
    print("3. Pop an item")
    print("4. Peek at top item")
    print("5. Check if stack is empty")
    print("6. Get stack size")
    print("7. Save stack to file")
    print("8. Load stack from file")
    print("9. Display stack")
    print("10. Clear stack")
    print("0. Exit")

    #Factory Method to create stack
    stack = StackFactory.create_stack()
    extended_stack = ExtendedStack()
    file_handler = FileHandler()

    while True:
        choice = input("\nEnter your choice (0-10): ")

        if choice == '0':
            print("Exiting program.")
            break

        elif choice == '1':
            stack = StackFactory.create_stack()
            print("New stack created.")

        elif choice == '2':
            item = input("Enter item to push: ")
            # Try to convert to number if possible
            try:
                item = float(item) if '.' in item else int(item)
            except ValueError:
                pass
            stack.push(item)
            print(f"Pushed: {item}")

        elif choice == '3':
            try:
                print(f"Popped: {stack.pop()}")
            except IndexError as e:
                print(f"Error: {e}")

        elif choice == '4':
            try:
                print(f"Top item: {stack.peek()}")
            except IndexError as e:
                print(f"Error: {e}")

        elif choice == '5':
            print("Stack is empty." if stack.is_empty() else "Stack is not empty.")

        elif choice == '6':
            print(f"Stack size: {stack.size()}")

        elif choice == '7':
            save_choice = input("Save to (1) default file or (2) custom file? ")
            if save_choice == '2':
                filename = input("Enter filename: ")
                custom_handler = FileHandler(filename)
                custom_handler.save_stack(stack)
                print(f"Stack saved to {filename}")
            else:
                file_handler.save_stack(stack)
                print(f"Stack saved to {file_handler.filename}")

        elif choice == '8':
            load_choice = input("Load from (1) default file or (2) custom file? ")
            if load_choice == '2':
                filename = input("Enter filename: ")
                if not os.path.exists(filename):
                    print(f"File {filename} does not exist!")
                    continue
                custom_handler = FileHandler(filename)
                custom_handler.load_stack(stack)
                print(f"Stack loaded from {filename}")
            else:
                if not os.path.exists(file_handler.filename):
                    print(f"File {file_handler.filename} does not exist!")
                    continue
                file_handler.load_stack(stack)
                print(f"Stack loaded from {file_handler.filename}")

        elif choice == '9':
            print(stack)

        elif choice == '10':
            stack.clear()
            print("Stack cleared.")

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    # Run unit tests if executed directly
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
main ()

...........
----------------------------------------------------------------------
Ran 11 tests in 0.020s

OK


Stack Class Demonstration
1. Create a new stack
2. Push an item
3. Pop an item
4. Peek at top item
5. Check if stack is empty
6. Get stack size
7. Save stack to file
8. Load stack from file
9. Display stack
10. Clear stack
0. Exit

Enter your choice (0-10): 0
Exiting program.
